In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation, Dropout, Convolution2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential

batch_size = 32

nb_epoch = 36


nb_classes = 2
image_shape = [64, 64, 3]

model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=image_shape))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
          optimizer='rmsprop',
          metrics=['accuracy'])

datagen = ImageDataGenerator(width_shift_range=.2, height_shift_range=.2, shear_range=0.05, zoom_range=.2,
                         fill_mode='nearest', rescale=1. / 255)
image_data_gen = datagen.flow_from_directory('Traffic_light', target_size=(64, 64), classes=['red', 'notred'],
                                         batch_size=batch_size)
model.fit_generator(image_data_gen, nb_epoch=nb_epoch, samples_per_epoch=image_data_gen.nb_sample)

model.save('light_classifier_model.h5')

Found 386 images belonging to 2 classes.
Epoch 1/36
386/386 [==============================] - 6s - loss: 0.8381 - acc: 0.5389     
Epoch 2/36
386/386 [==============================] - 6s - loss: 0.6103 - acc: 0.6684     
Epoch 3/36
386/386 [==============================] - 6s - loss: 0.5199 - acc: 0.7176     
Epoch 4/36
386/386 [==============================] - 6s - loss: 0.6011 - acc: 0.6477     
Epoch 5/36
386/386 [==============================] - 6s - loss: 0.5548 - acc: 0.7254     
Epoch 6/36
386/386 [==============================] - 6s - loss: 0.6085 - acc: 0.7435     
Epoch 7/36
386/386 [==============================] - 6s - loss: 0.4668 - acc: 0.7798     
Epoch 8/36
386/386 [==============================] - 6s - loss: 0.4562 - acc: 0.7720     
Epoch 9/36
386/386 [==============================] - 6s - loss: 0.4243 - acc: 0.8031     
Epoch 10/36
386/386 [==============================] - 6s - loss: 0.4822 - acc: 0.7642     
Epoch 11/36
386/386 [===========================